In [ ]:
#pip install pyhrv

In [ ]:
#   This filter is able to detect
# the principal artifacts encountered in such recording and to
# rebuild the R-R series by computing the most probable
# samples while keeping the real recording time

In [1]:
import pandas as pd

In [2]:
# Tipo 1: Missing
# R2 = 2RR1

# Tipo 2: Wrong detection
# RR1 = RR2 + RR3 (os dois seguintes são a soma do anterior correto)

# Tipo 3: Ectopic Beat
# 2RR1 = RR2 + RR3

In [3]:
def detection(df, wSize = 20):
    windowSize = wSize
    i = 1
    while True:
        bottomBound = i - 1
        upperBound  = windowSize + (i - 1)
        samplesWindow = df.iloc[bottomBound:upperBound]
        samplesWindow = samplesWindow[samplesWindow.Mark != 0]
        mean = samplesWindow.RR.mean()
        std  = samplesWindow.RR.std()

        bottomThreshold = mean - 2 * std
        upperThreshold  = mean + 2 * std

        for j in range(0, windowSize):
            RRi  = df.iloc[bottomBound + j].RR
            RR_1 = df.iloc[bottomBound + j - 1].RR
            RR1  = df.iloc[bottomBound + j + 1].RR

            if ((RRi >= bottomThreshold) and (RRi <= upperThreshold)):
                df['Mark'][bottomBound + j] = 1
            else:
                if ((RRi < bottomThreshold) and (RR1 > upperThreshold)):
                    df['Mark'][bottomBound + j] = 0
                elif ((RRi < 0.75 * RR_1) or (RR1 < 0.75 * RR_1)):
                    df['Mark'][bottomBound + j] = 0
                elif ((RRi > 1.75 * RR_1)):
                    df['Mark'][bottomBound + j] = 0

        firstUndefinedIndex = -1
        for j in range(0, windowSize):
            iMark  = df.iloc[bottomBound + j].Mark
            if (iMark == -1):
                firstUndefinedIndex = j
            else:
                if (firstUndefinedIndex != -1):
                    for k in range(firstUndefinedIndex, j):
                        df['Mark'][firstUndefinedIndex + k] = iMark
                    firstUndefinedIndex = -1

        i = i + 1

        if (windowSize != wSize):
            break
            
        if (upperBound > df.shape[0] - windowSize):
            windowSize = df.shape[0] - upperBound
            
    return df

In [4]:
def rebuild(df, start, end):
    s = -1
    f = -1
    for i in range(start, end):
        iMark = df.iloc[i].Mark
        if (iMark == 0):
            if (s != -1):
                s = i
            else:
                f = i

In [5]:
P = [
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\ParticipanteA_RR.csv',           350, '2019-11-06 11:15:22.809561',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\ParticipanteA_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\ParticipanteB_RR.csv',           50, '2019-11-06 11:15:22.809561',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\ParticipanteB_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\sound\ParticipanteA_RR.csv',  350, '2019-11-06 10:41:41.372607',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\sound\ParticipanteA_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\sound\ParticipanteB_RR.csv',  50, '2019-11-06 10:41:41.372607',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\sound\ParticipanteB_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\ParticipanteA\ParticipanteA_RR.csv',              350, '2019-11-06 11:37:38.271140',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\ParticipanteA\ParticipanteA_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\ParticipanteB\ParticipanteB_RR.csv',             50, '2019-11-06 11:37:38.271140',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\ParticipanteB\ParticipanteB_HR.csv'],

    
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x ParticipanteD\Multi\ParticipanteC_RR.csv',           350, '2019-11-06 11:36:43.641214',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x Pai\Multi\ParticipanteC_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x ParticipanteD\Multi\ParticipanteD_RR.csv',           50, '2019-11-06 11:36:43.641214',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x Pai\Multi\Pai_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x ParticipanteD\ParticipanteC\ParticipanteC_RR.csv',              350, '2019-11-06 11:24:05.014367',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x ParticipanteD\ParticipanteC\ParticipanteC_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x ParticipanteD\ParticipanteD\ParticipanteD_RR.csv',             50, '2019-11-06 11:24:05.014367',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x ParticipanteD\ParticipanteD\ParticipanteD_HR.csv'],
    
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteE x ParticipanteF\Multi\ParticipanteE_RR.csv',          50, '2019-11-06 12:53:24.837839‬',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteE x ParticipanteF\Multi\ParticipanteE_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteE x ParticipanteF\Multi\ParticipanteF_RR.csv',         350, '2019-11-06 12:53:24.837839‬',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteE x ParticipanteF\Multi\ParticipanteF_HR.csv'],
    
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteE x ParticipanteF\ParticipanteE\ParticipanteE_RR.csv',            50, '2019-11-06 12:06:58.992994',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteE x ParticipanteF\ParticipanteE\ParticipanteE_HR.csv'],
    [r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteE x ParticipanteF\ParticipanteF\ParticipanteF_RR.csv',           350, '2019-11-06 12:06:58.992994',          r'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteE x ParticipanteF\ParticipanteF\ParticipanteF_HR.csv']
    
]

In [10]:
for i in range(14):
    print(f"'{P[i][0]}', ")

'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\ParticipanteA_RR.csv', 
'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\ParticipanteB_RR.csv', 
'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\sound\ParticipanteA_RR.csv', 
'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\sound\ParticipanteB_RR.csv', 
'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\ParticipanteA\ParticipanteA_RR.csv', 
'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\ParticipanteB\ParticipanteB_RR.csv', 
'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x ParticipanteD\Multi\ParticipanteC_RR.csv', 
'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x ParticipanteD\Multi\ParticipanteD_RR.csv', 
'C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteC x Particip

In [7]:
for i in range(14):
    df = pd.read_csv(P[i][0])
    df['RR'] = df['RR Interval [ms]']
    df.drop(['RR Interval [ms]'], axis = 1, inplace = True)
    df['Mark'] = - 1

    #Mark
    # -1 -> Undefined
    # 0 -> Wrong
    # 1 -> Corret
    
    df = detection(df, wSize = 12)
    print(P[i][0])
    print(df[df.Mark == 0])

C:\Users\LucasMarquesGasparin\AppData\Local\Continuum\anaconda3\envs\TCC\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\LucasMarquesGasparin\AppData\Local\Continuum\anaconda3\envs\TCC\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\ParticipanteA_RR.csv
Empty DataFrame
Columns: [Time, RR, Mark]
Index: []
C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\ParticipanteB_RR.csv
Empty DataFrame
Columns: [Time, RR, Mark]
Index: []
C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\sound\ParticipanteA_RR.csv
Empty DataFrame
Columns: [Time, RR, Mark]
Index: []
C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\Multi\sound\ParticipanteB_RR.csv
Empty DataFrame
Columns: [Time, RR, Mark]
Index: []
C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\ParticipanteA\ParticipanteA_RR.csv
Empty DataFrame
Columns: [Time, RR, Mark]
Index: []
C:\Users\LucasMarquesGasparin\Desktop\Learning\TCC\ParticipanteA x ParticipanteB\ParticipanteB\ParticipanteB_RR.csv
Empty DataFrame
Columns: [Time, RR, Mark]
Index: []
C:\Users\Luc